<a href="https://colab.research.google.com/github/agrigoridou/Tokenization-Zipf-s-Law-N-gram-Models/blob/main/%CE%92_N_gram_Language_Models_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install nltk numpy

# Προετοιμασία των δεδομένων

Πρώτα, θα φορτώσουμε τα δεδομένα και θα προετοιμάσουμε τα αρχεία για την εκπαίδευση και αξιολόγηση:

In [45]:
import nltk
from nltk.corpus import treebank
from collections import Counter

# Φορτώνουμε τα δεδομένα
nltk.download('treebank')

# Φορτώνουμε τα πρώτα 150 αρχεία για εκπαίδευση και τα υπόλοιπα 49 για αξιολόγηση
train_files = treebank.fileids()[:150]
test_files = treebank.fileids()[150:]

# Λήψη των προτάσεων
train_sents = [sent for file in train_files for sent in treebank.sents(file)]
test_sents = [sent for file in test_files for sent in treebank.sents(file)]


[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Package treebank is already up-to-date!


# 1: Υπολογισμός perplexity στα κείμενα αξιολόγησης

In [46]:
def calculate_perplexity(model, sents, n=2):
    log_prob = 0
    total_ngrams = 0

    for sent in sents:
        if n == 2:
            ngrams_list = bigrams(sent)
        elif n == 3:
            ngrams_list = trigrams(sent)

        for ngram in ngrams_list:
            if n == 2:
                prob = model.get(ngram, 1e-6)  # Για bigram
            elif n == 3:
                prob = model.get(ngram, 1e-6)  # Για trigram

            log_prob += math.log(prob)
            total_ngrams += 1

    perplexity = math.exp(-log_prob / total_ngrams)
    return perplexity


## Υπολογισμός perplexity για τα μοντέλα (bigram και trigram)

In [57]:
import pandas as pd

perplexity_bigram_1 = calculate_perplexity(bigram_model_1, test_sents, n=2)
perplexity_bigram_2 = calculate_perplexity(bigram_model_2, test_sents, n=2)
perplexity_trigram_1 = calculate_perplexity(trigram_model_1, test_sents, n=3)
perplexity_trigram_2 = calculate_perplexity(trigram_model_2, test_sents, n=3)

# Δημιουργία πίνακα με τα αποτελέσματα
data = {
    "Model": ["Bigram k=1", "Bigram k=0.01", "Trigram k=1", "Trigram k=0.01"],
    "Perplexity": [perplexity_bigram_1, perplexity_bigram_2, perplexity_trigram_1, perplexity_trigram_2]
}

# Δημιουργία DataFrame από τα δεδομένα
df = pd.DataFrame(data)
df

,Model,Perplexity
0,Bigram k=1,48383.258496
1,Bigram k=0.01,20958.120746
2,Trigram k=1,538873.515246
3,Trigram k=0.01,320035.619599


# 2: Μετατροπή όλων των κειμένων σε πεζά

In [48]:
def preprocess_lowercase(sents):
    return [[token.lower() for token in sent] for sent in sents]


## Προετοιμασία πεζών δεδομένων

In [49]:
train_lower = preprocess_lowercase(train_sents)
test_lower = preprocess_lowercase(test_sents)

## Υπολογισμός perplexity για τα μοντέλα με πεζά γράμματα

In [58]:
import pandas as pd

# Υπολογισμός perplexity για τα lowercase μοντέλα (bigram και trigram)
perplexity_bigram_lower_1 = calculate_perplexity(bigram_model_1, test_lower, n=2)
perplexity_bigram_lower_2 = calculate_perplexity(bigram_model_2, test_lower, n=2)
perplexity_trigram_lower_1 = calculate_perplexity(trigram_model_1, test_lower, n=3)
perplexity_trigram_lower_2 = calculate_perplexity(trigram_model_2, test_lower, n=3)

# Δημιουργία πίνακα με τα αποτελέσματα
data_lower = {
    "Model": ["Bigram (Lowercase) k=1", "Bigram (Lowercase) k=0.01", "Trigram (Lowercase) k=1", "Trigram (Lowercase) k=0.01"],
    "Perplexity": [perplexity_bigram_lower_1, perplexity_bigram_lower_2, perplexity_trigram_lower_1, perplexity_trigram_lower_2]
}

# Δημιουργία DataFrame από τα δεδομένα
df_lower = pd.DataFrame(data_lower)

df_lower


,Model,Perplexity
0,Bigram (Lowercase) k=1,20016.401066
1,Bigram (Lowercase) k=0.01,6759.136604
2,Trigram (Lowercase) k=1,390205.901466
3,Trigram (Lowercase) k=0.01,179792.221192


#3: Αντικατάσταση ψηφίων με '#'

In [51]:
import re

def replace_digits_with_hash(sents):
    return [[re.sub(r'\d', '#', token) for token in sent] for sent in sents]


## Προετοιμασία δεδομένων με αντικατάσταση ψηφίων

In [52]:
train_hash = replace_digits_with_hash(train_sents)
test_hash = replace_digits_with_hash(test_sents)

## Υπολογισμός perplexity για τα μοντέλα με αντικατάσταση ψηφίων

In [59]:
import pandas as pd

# Υπολογισμός perplexity για τα hash δεδομένα (bigram και trigram)
perplexity_bigram_hash_1 = calculate_perplexity(bigram_model_1, test_hash, n=2)
perplexity_bigram_hash_2 = calculate_perplexity(bigram_model_2, test_hash, n=2)
perplexity_trigram_hash_1 = calculate_perplexity(trigram_model_1, test_hash, n=3)
perplexity_trigram_hash_2 = calculate_perplexity(trigram_model_2, test_hash, n=3)

# Δημιουργία πίνακα με τα αποτελέσματα
data_hash = {
    "Model": ["Bigram (Hash) k=1", "Bigram (Hash) k=0.01", "Trigram (Hash) k=1", "Trigram (Hash) k=0.01"],
    "Perplexity": [perplexity_bigram_hash_1, perplexity_bigram_hash_2, perplexity_trigram_hash_1, perplexity_trigram_hash_2]
}

# Δημιουργία DataFrame από τα δεδομένα
df_hash = pd.DataFrame(data_hash)

df_hash


,Model,Perplexity
0,Bigram (Hash) k=1,79285.109869
1,Bigram (Hash) k=0.01,39430.240567
2,Trigram (Hash) k=1,649137.763097
3,Trigram (Hash) k=0.01,446736.027853


# 4: Δημιουργία νέων προτάσεων

In [54]:
def generate_sentence(model, n=2, max_len=20):
    sentence = ['<BOS>']
    while len(sentence) < max_len:
        last_word = sentence[-1]
        possible_next_words = {key[1] for key in model if key[0] == last_word}

        if not possible_next_words:
            break

        next_word = random.choices(list(possible_next_words), weights=[model.get((last_word, word), 1e-6) for word in possible_next_words])[0]
        sentence.append(next_word)

        if next_word == '<EOS>':
            break

    return sentence

## Δημιουργία νέων προτάσεων

In [60]:
import pandas as pd

# Δημιουργία νέων προτάσεων από τα μοντέλα bigram και trigram
new_sentence_1 = generate_sentence(bigram_model_1, n=2)
new_sentence_2 = generate_sentence(trigram_model_1, n=3)
new_sentence_3 = generate_sentence(bigram_model_2, n=2)

# Δημιουργία πίνακα με τα αποτελέσματα
data_sentences = {
    "Model": ["Bigram Model k=1", "Trigram Model k=1", "Bigram Model k=0.01"],
    "Generated Sentence": [new_sentence_1, new_sentence_2, new_sentence_3]
}

# Δημιουργία DataFrame από τα δεδομένα
df_sentences = pd.DataFrame(data_sentences)

df_sentences


,Model,Generated Sentence
0,Bigram Model k=1,"[<BOS>, in, tokyo, giants, ., <EOS>]"
1,Trigram Model k=1,"[<BOS>, revenue, from, diamond, creek, ., was,..."
2,Bigram Model k=0.01,"[<BOS>, some, <UNK>, long, bond, due, <UNK>, o..."
